# VI - Tensorboard Visualization
This notebook will show how to create an SSH tunnel from the machine running the Notebook to the compute node of a task that is running or has run a task that has generated [Tensorboard](https://www.tensorflow.org/get_started/summaries_and_tensorboard) summary compatible output.

**NOTE:** This notebook cannot be run on Azure notebooks due to restrictions. Please run this notebook locally. If you are running this notebook on Windows, please ensure you have `ssh.exe` in your `%PATH%`. You can download OpenSSH binaries for Windows [here](https://github.com/PowerShell/Win32-OpenSSH/releases).

* [Setup](#section1)
* [Configure job](#section2)
* [Submit job](#section3)
* [Delete job](#section4)

<a id='section1'></a>

## Setup

Create a simple alias for Batch Shipyard

In [ ]:
%alias shipyard SHIPYARD_CONFIGDIR=config python $HOME/batch-shipyard/shipyard.py %l

Check that everything is working

In [ ]:
shipyard --version

Get some variables stored in the Setup notebook:

In [ ]:
import json

def read_json(filename):
    with open(filename, 'r') as infile:
        return json.load(infile)
    
account_info = read_json('account_information.json')

IMAGE_NAME = account_info['IMAGE_NAME']
STORAGE_ALIAS = account_info['STORAGE_ALIAS']

<a id='section2'></a>
## Configure job
The following will be similar to the [Single GPU Training](02_Single_GPU_Training.ipynb) notebook from earlier.

For the `jobs` configuration, we will add `--logdir=tensorboard_logs` as a parameter to generate the Tensorboard summary log data during the run.

In [ ]:
TASK_ID = 'run_cifar10' # This should be changed per task

JOB_ID = 'cntk-train-tensorboard-job'

COMMAND = 'bash -c "source /cntk/activate-cntk; python -u ConvNet_CIFAR10.py --datadir $AZ_BATCH_NODE_SHARED_DIR/data --tensorboard_logdir tensorboard_logs"'

jobs = {
    "job_specifications": [
        {
            "id": JOB_ID,
            "tasks": [
                {
                    "id": TASK_ID,
                    "image": IMAGE_NAME,
                    "remove_container_after_exit": True,
                    "command": COMMAND,
                    "gpu": True,
                    "resource_files": [
                        {
                            "file_path": "ConvNet_CIFAR10.py",
                            "blob_source": "https://batchshipyardexamples.blob.core.windows.net/code/ConvNet_CIFAR10.py",
                            "file_mode":'0777'
                        }
                    ],
                    "output_data": {
                        "azure_storage": [
                            {
                                "storage_account_settings": STORAGE_ALIAS,
                                "container": "output",
                                "source": "$AZ_BATCH_TASK_WORKING_DIR/Models"
                            },
                        ]
                    },
                }
            ],
        }
    ]
}

Write the jobs configuration to the `jobs.json` file:

In [ ]:
import json
import os

def write_json_to_file(json_dict, filename):
    """ Simple function to write JSON dictionaries to files
    """
    with open(filename, 'w') as outfile:
        json.dump(json_dict, outfile)

write_json_to_file(jobs, os.path.join('config', 'jobs.json'))
print(json.dumps(jobs, indent=4, sort_keys=True))

<a id='section3'></a>

## Submit job
Check that everything is ok with our pool before we submit our jobs


In [ ]:
shipyard pool listnodes

Now that we have confirmed everything is working we can execute our job using the command below. Note that we'll not be using the `--tail` option so that the command completes and we can tunnel to Tensorboard concurrently as the task is executing.

In [ ]:
shipyard jobs add

Run the Batch Shipyard command to instantiate a Tensorboard instance and create an SSH tunnel. The following cell should not return immediately if it is working. Browse to the Tensorboard URL output by the command (which will not be output in the notebook since it is a blocking call), which is http://localhost:6006/

**Notes:**
1. The Tensorboard instance may take some time to start since this pool does not have the TensorFlow Docker image pre-loaded.
2. You will need to manually interrupt the kernel once you are done with your Tensorboard visualization.

In [ ]:
shipyard jobs listtasks

In [ ]:
shipyard misc tensorboard --jobid $JOB_ID --taskid $TASK_ID -y

<a id='section4'></a>

## Delete job

To delete the job use the command below. Just be aware that this will get rid of all the files created by the job and tasks.

In [ ]:
shipyard jobs del -y --termtasks --wait

In [ ]:
shipyard pool del -y --wait